# Importing the libraries

In [439]:
import pandas as pd
import time
import numpy as np
import math
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.sparse import csr_matrix, hstack, vstack

# Feature methods

In [440]:
# Function to calculate Count of Variables
def count_variables(script):
    # Extract variable declarations using a regular expression
    variable_declarations = re.findall(r'\b(?:Dim|Private|Public|Static)\s+(\w+)', script)
    
    # Return the count of distinct variable declarations divided by the script length
    return len(set(variable_declarations)) / len(script) if len(script) > 0 else 0

# Function to add Count of Variables column
def add_count_of_variables_column(dataframe):
    return dataframe['vba_code'].apply(count_variables)


In [441]:
# Function to calculate Average Variable Assignment Length
def calculate_avg_variable_assignment_length(text):
    # Extract variable assignments using a regular expression
    variable_assignments = re.findall(r'\b(?:Set|Dim)\s+(\w+)\s*=\s*(".+?"|\w+)', text)

    # Calculate the average length of string variables
    total_length = sum(len(value) for _, value in variable_assignments if '"' in value)
    count = sum(1 for _, value in variable_assignments if '"' in value)

    # Avoid division by zero
    if count == 0:
        return 0

    avg_length = total_length / count
    return avg_length

def add_avg_variable_assignment_length_column(dataframe):
    return dataframe['vba_code'].apply(calculate_avg_variable_assignment_length)

In [442]:
def mal_wor(text):
    unauthorized_patterns = ['Emoji','"Hacked!"','UserVersion',"'donwload",'Private Sub Sample()','On Error Resume Next']
    authorized = ['Private Sub ComboBox11_DropButtonClick()']
    for pattern in authorized:
        if pattern in text:
            return 0  # Detected unauthorized access

    for pattern in unauthorized_patterns:
        if pattern in text:
            return 1  # Detected unauthorized access

    return 0  # No unauthorized access detected

def add_mal_words_column(dataframe):
    return dataframe['vba_code'].apply(mal_wor)

In [443]:
def check_hexadecimal_encoding(text):
    return 1 if re.search(r'\b[0-9a-fA-F]+\b', text) else 0

def check_base64_encoding(text):
    return 1 if re.search(r'\b(?:[A-Za-z0-9+/]{4})*(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=)?\b', text) else 0

def add_check_hexadecimal_encoding_column(dataframe):
    return dataframe['vba_code'].apply(check_hexadecimal_encoding)

def add_check_base64_encoding_column(dataframe):
    return dataframe['vba_code'].apply(check_base64_encoding)


# Add columns to the df

In [444]:
 
df_unique = pd.read_csv('/Users/AmitBer/Downloads/macro_competition/train_dataset.csv', encoding='utf-16-le')



df_transposed = df_unique.T

# Remove duplicate rows
df_transposed_unique = df_transposed.drop_duplicates()

# Transpose the DataFrame back
df = df_transposed_unique.T


# Write the unique data back to a new CSV file
df.to_csv('unique_file.csv', index=False)
# Split the dataset into 'malicious' and 'white,' and shuffle the data
malicious = df[df['label'] == 'mal']
white = df[df['label'] == 'white']
merged_data = pd.concat([malicious, white], axis=0)

# Training model

In [445]:
# Prepare features (X) and labels (y)
X = merged_data['vba_code']
y = merged_data['label']

In [446]:

# Add number of variables feature
number_of_variables = np.array(add_count_of_variables_column(df))

avg_variable_assignment_length = np.array(add_avg_variable_assignment_length_column(df))

mal_words = np.array(add_mal_words_column(df))

check_hexadecimal_encodingn = np.array(add_check_base64_encoding_column(df))

check_base64_encodingn = np.array(add_check_hexadecimal_encoding_column(df))


In [447]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='utf-8', decode_error='ignore', stop_words='english')

# Fit and transform the data
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [448]:
# Combine the matrices horizontally
X_combined = hstack([X_tfidf,
                     csr_matrix(number_of_variables.reshape(-1,1)),
                     csr_matrix(avg_variable_assignment_length.reshape(-1,1)),
                     csr_matrix(mal_words.reshape(-1,1)),
                     csr_matrix(check_base64_encodingn.reshape(-1,1)), 
                     csr_matrix(check_hexadecimal_encodingn.reshape(-1,1))])

In [449]:
start_time = time.time()
classifier = RandomForestClassifier(n_estimators=58, random_state=42, min_samples_leaf=1)
classifier.fit(X_combined, y)
elapsed_time = time.time() - start_time

# Print the training time
print(f"Classifier training time: {elapsed_time:.2f} seconds")

Classifier training time: 64.08 seconds


In [450]:
import pickle

# Save the model
with open('top_class.pkl', 'wb') as f:
    pickle.dump(classifier, f)
    
with open('vec.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)      

In [451]:
# Make predictions on the entire dataset
predictions = classifier.predict(X_combined)

# Evaluate the classifier on the training set
accuracy = accuracy_score(y, predictions)
conf_matrix = confusion_matrix(y, predictions)
class_report = classification_report(y, predictions)

# Print the results for the training set
print(f"Training Accuracy: {accuracy}")
print(f"Training Confusion Matrix:\n{conf_matrix}")
print(f"Training Classification Report:\n{class_report}")

Training Accuracy: 0.9997177621675866
Training Confusion Matrix:
[[15808     2]
 [    7 16071]]
Training Classification Report:
              precision    recall  f1-score   support

         mal       1.00      1.00      1.00     15810
       white       1.00      1.00      1.00     16078

    accuracy                           1.00     31888
   macro avg       1.00      1.00      1.00     31888
weighted avg       1.00      1.00      1.00     31888



# Validation

In [452]:
# Read the validation dataset
validation_df = pd.read_csv('/Users/AmitBer/Downloads/macro_competition/validation_dataset.csv', encoding='utf-16-le')

In [453]:
# Extract features and labels from the validation set
X_validation = validation_df['vba_code']
y_validation = validation_df['label']

In [454]:
# Transform the validation data using the same vectorizer
X_validation_tfidf = tfidf_vectorizer.transform(X_validation)


In [455]:
# Add number of variables feature to the validation
number_of_variables_validation = np.array(add_count_of_variables_column(validation_df))

# Add average variable assignment length feature to the validation
avg_variable_assignment_length_validation = np.array(add_avg_variable_assignment_length_column(validation_df))

check_on_error_resume_validation = np.array(add_mal_words_column(validation_df))

check_hexadecimal_encodingn_validation = np.array(add_check_base64_encoding_column(validation_df))

check_base64_encodingn_validation = np.array(add_check_hexadecimal_encoding_column(validation_df))


In [456]:
# Combine the matrices horizontally
X_validation_combined = hstack([X_validation_tfidf,
                        csr_matrix(number_of_variables_validation.reshape(-1,1)),
                        csr_matrix(avg_variable_assignment_length_validation.reshape(-1,1)),
                        csr_matrix(check_on_error_resume_validation.reshape(-1,1)),
                        csr_matrix(check_hexadecimal_encodingn_validation.reshape(-1,1)),
                        csr_matrix(check_base64_encodingn_validation.reshape(-1,1))])

In [457]:
# Make predictions on the validation set and evaluate the classifier's performance
predictions_validation = classifier.predict(X_validation_combined)
validation_df['generated_label'] = predictions_validation

# Results

In [458]:
# Print the results for the validation set
accuracy_validation = accuracy_score(y_validation, predictions_validation)
conf_matrix_validation = confusion_matrix(y_validation, predictions_validation)
class_report_validation = classification_report(y_validation, predictions_validation)

print(f"Validation Accuracy: {accuracy_validation}")
print(f"Validation Confusion Matrix:\n{conf_matrix_validation}")
print(f"Validation Classification Report:\n{class_report_validation}")

Validation Accuracy: 0.9953899708345093
Validation Confusion Matrix:
[[5273   47]
 [   2 5307]]
Validation Classification Report:
              precision    recall  f1-score   support

         mal       1.00      0.99      1.00      5320
       white       0.99      1.00      1.00      5309

    accuracy                           1.00     10629
   macro avg       1.00      1.00      1.00     10629
weighted avg       1.00      1.00      1.00     10629



In [459]:
true_negative_indices = (predictions_validation == 'white') & (y_validation == 'mal')
print("\nTrue Negative- Mal but was predicted as white Rows in Original CSV:")
print(validation_df.loc[true_negative_indices])


True Negative- Mal but was predicted as white Rows in Original CSV:
      label                                           vba_code generated_label
50      mal                                                  b           white
63      mal  Function AAA()\nEnd Function\nIf 1 <> 1 Then\n...           white
421     mal  Sub OYwrUVCJckZuLvRBMSOZrFN()\n\nDim tudfTFyTf...           white
486     mal  Function E5IQj(ByVal O8QGyoRH9 As String, ByVa...           white
1479    mal  Sub Document_Open()\nIf 23 < 153 Then\n' rRXLq...           white
1881    mal  Sub KC()\nDim Temp(3) As String\nOn Error GoTo...           white
1922    mal  Public Function HexToString(ByVal RltjOJKINyry...           white
2040    mal  Function AAA()\nEnd Function\nIf 1 <> 1 Then\n...           white
2081    mal  Sub AutoOpen()\n    Application.Run "khhzrzr"\...           white
2092    mal  Private Sub workbook_open()\nfWH7voI76HLe.f_29...           white
2138    mal  Sub backlash()\nDim hinny As Variant\nDim mel ...

In [438]:
dft = pd.read_excel('/Users/AmitBer/Downloads/test.xlsx')
dft.to_csv('test_prediction.csv')
X_validation_t = dft['vba_code']
X_validation_tfidf = tfidf_vectorizer.transform(X_validation_t)


# Add number of variables feature to the validation
number_of_variables_validation = np.array(add_count_of_variables_column(dft))

# Add average variable assignment length feature to the validation
avg_variable_assignment_length_validation = np.array(add_avg_variable_assignment_length_column(dft))

check_on_error_resume_validation = np.array(add_check_on_error_resume_next_column(dft))

check_hexadecimal_encodingn_validation = np.array(add_check_base64_encoding_column(dft))

check_base64_encodingn_validation = np.array(add_check_hexadecimal_encoding_column(dft))

# Combine the matrices horizontally
X_validation_combined_t = hstack([X_validation_tfidf,
                                  csr_matrix(number_of_variables_validation.reshape(-1,1)),
                                  csr_matrix(avg_variable_assignment_length_validation.reshape(-1,1)),
                                  csr_matrix(check_on_error_resume_validation.reshape(-1,1)),
                                  csr_matrix(check_hexadecimal_encodingn_validation.reshape(-1,1)),
                                  csr_matrix(check_base64_encodingn_validation.reshape(-1,1))])


predictions_validation_t = classifier.predict(X_validation_combined_t)

dft.rename(columns={'vba_code': 'prediction'}, inplace=True)
dft['prediction'] = predictions_validation_t

In [3064]:
dft

,prediction
0,white
1,white
2,mal
3,white
4,white
...,...
10625,white
10626,mal
10627,white
10628,mal
